In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

In [2]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [3]:
imageSize = 28
numChannels = 3
numLabels = 2
batchSize = 20
numEpochs = 3
trainSize = 100


# declare graph operations
reset_graph()
# Inplementing Convolution layer
def convActivation(inputData, inpOutShape=[5,5,3,12], name="convLayer"):
    kernelY, kernelX, numImp, numOut = inpOutShape
    with tf.name_scope(name):
        with tf.variable_scope(name):
            w = tf.get_variable(dtype=tf.float32, 
                                shape=inpOutShape, 
                                initializer=tf.random_normal_initializer(
                                    mean=0, stddev=0.1, seed = 9743),
                                name="convWghts")
            b = tf.get_variable(dtype=tf.float32,
                               shape=[numOut],
                               initializer=tf.constant_initializer(1.0),
                               name="convBias")

            conv = tf.nn.conv2d(inputData, w, strides=[1,1,1,1], padding="SAME")
            act = tf.nn.relu(conv + b)
            pool = tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        
            return pool
    
# Implement fully connected layer
def fcLayer(inputData, inpOutShape, name="fcLayer"):
    numInp, numOut = inpOutShape
    with tf.name_scope(name):
        with tf.variable_scope(name):
            w = tf.get_variable(dtype=tf.float32,
                               shape=inpOutShape,
                               initializer=tf.random_normal_initializer(
                                   mean=0, stddev=0.1, seed=2349),
                               name="fcWghts")
            b = tf.get_variable(dtype=tf.float32,
                               shape=[numOut],
                               initializer=tf.constant_initializer(1.0),
                               name="fcBias")
            convToFc1 = tf.matmul(inputData, w) + b

            return tf.nn.relu(convToFc1)
    
# Build the forward feed network

x = tf.placeholder(tf.float32, shape=[None, imageSize, imageSize, numChannels], name='xInp')
y = tf.placeholder(tf.float32, shape=[None, numLabels], name="xLabels")
print (x.get_shape().as_list(), y.get_shape().as_list())
conv1 = convActivation(inputData=x,
                      inpOutShape=[5,5,3,12],
                      name = "conv1")
conv2 = convActivation(inputData=conv1,
                      inpOutShape=[5,5,12,12],
                      name = "conv2")

shapeY, shapeX, depth = conv2.get_shape().as_list()[1:4]
flattenedShape = shapeY * shapeX * depth
conv2Flattened = tf.reshape(conv2, [-1, flattenedShape])
fcOut1 = fcLayer(inputData=conv2Flattened,
                inpOutShape=[flattenedShape, 28],
                name="fcLayer1")
fcOut2 = fcLayer(inputData=fcOut1,
                inpOutShape=[28,numLabels],
                name="fcLayer2")

# Now we send our input to the softmax function to calculate the cross entropy loss
with tf.name_scope("trainLoss"):
    loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=fcOut2, labels=y)
    )

# Implement the optimizer
with tf.name_scope("trainOpt"):
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

# finally we calculate the accuracy
with tf.name_scope("Accuracy"):
    correct_prediction = tf.equal(tf.argmax(fcOut2,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

[None, 28, 28, 3] [None, 2]


In [5]:
[op for op in tf.get_default_graph().get_operations()]

[<tf.Operation 'xInp' type=Placeholder>,
 <tf.Operation 'xLabels' type=Placeholder>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal/shape' type=Const>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal/mean' type=Const>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal/stddev' type=Const>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal/RandomStandardNormal' type=RandomStandardNormal>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal/mul' type=Mul>,
 <tf.Operation 'conv1/convWghts/Initializer/random_normal' type=Add>,
 <tf.Operation 'conv1/convWghts' type=VariableV2>,
 <tf.Operation 'conv1/convWghts/Assign' type=Assign>,
 <tf.Operation 'conv1/convWghts/read' type=Identity>,
 <tf.Operation 'conv1/convBias/Initializer/Const' type=Const>,
 <tf.Operation 'conv1/convBias' type=VariableV2>,
 <tf.Operation 'conv1/convBias/Assign' type=Assign>,
 <tf.Operation 'conv1/convBias/read' type=Identity>,
 <tf.Operation 'conv1/conv1/Conv2D' type=Conv2D>,
 

In [4]:
summaryOutput = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/TensorFlowOPS"
# Here we build fake mnist data
def fake_data(trainSize, numFeatures):
    """Generate a fake dataset that matches the dimensions of MNIST."""
    data = np.random.rand(
        trainSize, imageSize, imageSize, 3)
    labels = np.zeros(shape=(trainSize,), dtype=np.int64)
    for image in xrange(trainSize):
        label = image % 2
#         data[image, 0] = label - 0.5
        labels[image] = label
    return data, labels

trainData, trainLabels = fake_data(trainSize=trainSize, numFeatures=3)

trainLabels = np.eye(numLabels)[trainLabels]
numSteps = int((numEpochs * trainSize)) // batchSize
# Capture the remainder if in case the trainSize is not ezactly divisible by batchSize.  If such is the case then we may miss some training set. 
# For example, if trainingSize = 11 and batchSize=2 them since 10%2 = 0, but 11%2 = 1. So when we itereate over a batch of 2 then for every epoch 
# we miss one trainig example
remainder = trainSize % batchSize  
sess = tf.Session()
with sess.as_default():
    sess.run(tf.global_variables_initializer())
#     print ('Total number of Steps are: ', numSteps)
    for step in np.arange(numSteps):
        offset = (step * batchSize) % (trainSize-remainder)
#         print ('(Step * batchSize) modulus (trainSize - remainder) = offset (%s * %s) modulus (%s - %s) = %s : '%(str(step), str(batchSize), str(trainSize), str(remainder), str(offset)))
        if offset == (trainSize - remainder - batchSize):
            batchData = trainData[offset:(offset+batchSize+remainder),:]
            batchLabels = trainLabels[offset:(offset+batchSize+remainder),:]
        else:
            batchData = trainData[offset:(offset+batchSize),:]
            batchLabels = trainLabels[offset:(offset+batchSize),:]
#         print (batchData.shape)
        feedDict = {x:batchData, y:batchLabels}
        fc_out, lessCE, acc, _ = sess.run([fcOut2,loss,accuracy,optimizer],feed_dict=feedDict)
        print (acc)
        
    # Here we store the session into the Disk
    writer = tf.summary.FileWriter(summaryOutput)
    writer.add_graph(sess.graph)
    writer.close()



0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5
0.5


In [28]:
data = np.ndarray(
        shape=(20, 28, 28, 3),
        dtype=np.float32)

In [31]:
data.shape

(20, 28, 28, 3)